# Basic ETL With Pandas

ETL stands for "Extract Transform Load" and it's a blanket term for a common type of software process. Specifically a process where:

1. You have data in some source format that you need to "extract"
2. The data doesn't perfectly match the format you want the data to be in, so you "transform" it
3. The data needs to be stored once again, so you "load" it into the new storage system or format.



In [7]:
# In this case our data starts in a CSV, which is pretty transportable...
import pandas as pd
path_to_ny_sales = 'nyc-property-data/nyc-rolling-sales.csv'
sales_df = pd.read_csv(path_to_ny_sales)
sales_df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


## Lets perform a few transformations:

1. Replace borough with the actual name of the borough as a string.
2. Remove some columns that we don't care about for whatever reason.
3. Drop rows from numerical columns with non-numerical or otherwise missing data.
4. Add a column that indicates "residential" "commercial" or "mixed use" based on the number of units.

In [8]:
# 1: use the map feature to replace values in columns with an alternate
sales_df['BOROUGH'] = sales_df['BOROUGH'].map({
    1 : 'Manhattan',
    2 : 'Bronx',
    3 : 'Brooklyn',
    4 : 'Queens',
    5 : 'Staten Island',
})

sales_df['BOROUGH']

0            Manhattan
1            Manhattan
2            Manhattan
3            Manhattan
4            Manhattan
             ...      
84543    Staten Island
84544    Staten Island
84545    Staten Island
84546    Staten Island
84547    Staten Island
Name: BOROUGH, Length: 84548, dtype: object

In [9]:
# 2: Drop some of the columns
columns_to_drop = [
    'Unnamed: 0',
    'TAX CLASS AT PRESENT',
    'BLOCK',
    'LOT',
    'EASE-MENT',
    'BUILDING CLASS AT PRESENT',
    'TAX CLASS AT TIME OF SALE',
    'BUILDING CLASS AT TIME OF SALE',
    'BUILDING CLASS CATEGORY'
]

sales_df = sales_df.drop(columns=columns_to_drop)

sales_df.head()

,BOROUGH,NEIGHBORHOOD,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE
0,Manhattan,ALPHABET CITY,153 AVENUE B,,10009,5,0,5,1633,6440,1900,6625000,2017-07-19 00:00:00
1,Manhattan,ALPHABET CITY,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,-,2016-12-14 00:00:00
2,Manhattan,ALPHABET CITY,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,-,2016-12-09 00:00:00
3,Manhattan,ALPHABET CITY,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,3936272,2016-09-23 00:00:00
4,Manhattan,ALPHABET CITY,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,8000000,2016-11-17 00:00:00


In [10]:
# 3: Convert columns and drop na values
columns_to_convert = [
    'LAND SQUARE FEET',
    'GROSS SQUARE FEET',
    'SALE PRICE',
    'YEAR BUILT'
]

for column_name in columns_to_convert:
    sales_df[column_name] = pd.to_numeric(sales_df[column_name], errors='coerce')
    sales_df = sales_df[sales_df[column_name].notna()]
    
sales_df.describe()

,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE
count,48244.000000,48244.000000,48244.000000,48244.000000,4.824400e+04,4.824400e+04,48244.000000,4.824400e+04
mean,10939.681328,2.566537,0.249171,2.834632,3.358117e+03,3.669753e+03,1827.765173,1.153281e+06
std,1072.752086,17.465481,10.988072,20.750418,3.143590e+04,2.947491e+04,464.361153,1.340131e+07
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,10469.000000,1.000000,0.000000,1.000000,1.413000e+03,8.280000e+02,1920.000000,8.042000e+04
50%,11217.000000,1.000000,0.000000,1.000000,2.140000e+03,1.620000e+03,1931.000000,4.800000e+05
75%,11237.000000,2.000000,0.000000,2.000000,3.071000e+03,2.520000e+03,1961.000000,8.300000e+05
max,11694.000000,1844.000000,2261.000000,2261.000000,4.228300e+06,3.750565e+06,2017.000000,2.210000e+09


In [12]:
# 4: create a column based on data in the other columns
# Specifically, check if this property is residential, commercial, or mixed use
def check_building_type(row):
    if row['COMMERCIAL UNITS'] > 0 and row['RESIDENTIAL UNITS'] > 0:
        return "MIXED USE"
    elif row['COMMERCIAL UNITS'] > 0:
        return "COMMERCIAL"
    elif row['RESIDENTIAL UNITS']:
        return "RESIDENTIAL"
    else:
        return "UNKNOWN - NO UNITS"
    
# axis=1 means apply the function to rows
# axis=0 would mean apply the function to the columns
sales_df['BUILDING TYPE'] = sales_df.apply(check_building_type, axis=1) 

sales_df.head()

,BOROUGH,NEIGHBORHOOD,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,SALE PRICE,SALE DATE,BUILDING TYPE
0,Manhattan,ALPHABET CITY,153 AVENUE B,,10009,5,0,5,1633.0,6440.0,1900,6625000.0,2017-07-19 00:00:00,RESIDENTIAL
3,Manhattan,ALPHABET CITY,154 EAST 7TH STREET,,10009,10,0,10,2272.0,6794.0,1913,3936272.0,2016-09-23 00:00:00,RESIDENTIAL
4,Manhattan,ALPHABET CITY,301 EAST 10TH STREET,,10009,6,0,6,2369.0,4615.0,1900,8000000.0,2016-11-17 00:00:00,RESIDENTIAL
6,Manhattan,ALPHABET CITY,210 AVENUE B,,10009,8,0,8,1750.0,4226.0,1920,3192840.0,2016-09-23 00:00:00,RESIDENTIAL
9,Manhattan,ALPHABET CITY,629 EAST 5TH STREET,,10009,24,0,24,4489.0,18523.0,1920,16232000.0,2016-11-07 00:00:00,RESIDENTIAL


# Load

There are alternate mechanisms in Python for loading data into any number of databases and other storage mechanisms. For now, lets just load to a file since we don't assume knowledge of SQL or anything else for this class.

In [14]:
# Pandas makes it very easy to write to a CSV just as it does for reading.
# See more options in the docs: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html
sales_df.to_csv('nyc-property-data/transformed_nyc_housing.csv', index=False)

# An Important Note:

ETL jobs ideally will be super easy to run, and potentially even automated in most situations. As a result, Jupyter Notebook is not the most ideal format for this kind of work. In a real world project I would:

1. Write the transformations and confirm they work in a Jupyter notebook during the prototyping phase.
2. Move the transformations to a single script.
3. (probably) Write some tests to ensure the ETL job performs as expected.

The script for step 2 of this transformation is in the code repository with the name: 